In [1]:
import tensorflow as tf
import os
import requests
import io
import PIL
import PIL.Image
import math
import pathlib
from tensorflow.keras import layers

In [2]:
batch_size = 8 # 每⼀批所处理的图⽚数量
img_height = 256 # 图⽚⾼度，单位为像素
img_width = 256 # 图⽚宽度，单位为像素

train_dir = './datasets/flower_photos'

TF_SERVING_BASE_URL = 'http://localhost:8501/'

In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
train_dir,
validation_split=0.2, # 设定验证集⽐例
subset="training",
seed=123,
#image_size=(img_height, img_width),
batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


In [4]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
train_dir,
validation_split=0.2, # 设定验证集⽐例
subset="validation",
seed=123,
#image_size=(img_height, img_width),
batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [5]:
print(train_ds.class_names)
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
(8, 256, 256, 3)
(8,)


In [6]:
num_classes = 5
model = tf.keras.Sequential([ # 根据需要调整模型结构
layers.experimental.preprocessing.Resizing(img_height, img_width),
tf.keras.layers.experimental.preprocessing.Rescaling(1. / 255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
   
])

In [7]:
model.compile(
optimizer='adam',
loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=['accuracy'])

In [8]:
model.fit(
train_ds,
validation_data=val_ds,
epochs=1
)
print(model.evaluate(val_ds))

92/92 [==============================] - 6s 62ms/step - loss: 1.0600 - accuracy: 0.6172
[1.0599966049194336, 0.6171662211418152]


In [9]:
tf.keras.models.save_model(
model,
'./models/image/1/', # ./models为tensorflow-serving的模型根⽬录
overwrite=True,
include_optimizer=True,
save_format=None,
signatures=None,
options=None
)

INFO:tensorflow:Assets written to: ./models/image/1/assets


In [10]:
def test_image_model(test_dir, code, batch_size=10): 
  imgs = []
  codes = []
  imgdir = os.path.join(test_dir, str(code)) 
  for i in pathlib.Path(imgdir).glob('./*.jpg'): 
    img = PIL.Image.open(i)
    pixels = np.array(img)
    imgs.append(pixels.tolist()) 
  for i in range(int(math.ceil(len(imgs)/batch_size))):
    req_data = json.dumps({
'     inputs': imgs[i*batch_size:(i+1)*batch_size],
    }) 
    #http://localhost:8501/v1/models/fashion_mnist:predict
    response = requests.post(TF_SERVING_BASE_URL+'v1/models/image/versions/1:predict', # 根据部署地址填写
    data=req_data,
    headers={"content-type": "application/json"})
    if response.status_code != 200:
      raise RuntimeError('Request tf-serving failed: ' + response.text)
    resp_data = json.loads(response.text) 
    if 'outputs' not in resp_data \
         or type(resp_data['outputs']) is not list:
      raise ValueError('Malformed tf-serving response')
    codes.extend(np.argmax(resp_data['outputs'], axis=1).tolist())
    return codes

In [11]:
codes = test_image_model(val_ds, 0)
print('类别0的准确率', 1 - round(np.sum(codes)/len(codes),4))
codes = test_image_model(val_ds, 1)
print('类别1的准确率', round(np.sum(codes)/len(codes),4))

TypeError: expected str, bytes or os.PathLike object, not BatchDataset